# ロジスティック回帰
- 「回帰」という名前ではあるが分類問題を解くものである
- このアルゴリズムを実装していく

# 2値分類

ここからは出力ベクトル$y$は連続値から0もしくは1の値となる。
$$ y \in \{0, 1\}$$
このとき0は陰性、1は陽性などとして表現されるがいずれも定まった表現は無いとのこと。

このように二つの分類を考えることを「二値分類問題」と呼ぶ。

今までのように線形回帰を使ってこの問題を解く場合にはあらゆる予測結果をマッピングしておいて、それが0.5を越えれば1、越えなければ0と分類するやり方が一つ考えられるだろう。

しかしながらこの方法ではこの問題を上手く解くことができない。その原因は多くの問題が非線形であることに起因する。

## ルールベースではなくファジーに

唐突だが以下の問いに対して皆さんはどのように答えるだろうか？

"身長のデータから「背が高い人」と「背が低い人」の二つに分類できないか？"

咄嗟に思いつくアプローチは「ある閾値、たとえば165cm、に基づいて判断する」だろう。しかしながらこれは適切な手法と言えるだろうか？

この手法に基づくならば164.9cmは背が低い人に分類されてしまうわけだがそれはナンセンスだろう。165cmの人とはわずか0.1cmしか違わないのに・・・。

これはまさに「機械的」、もっと人間味の溢れる分け方はないものだろうか？そもそも我々が取り扱う様々な概念、「暑い」「寒い」「辛い」「甘い」・・・などは非常に曖昧であり一律に閾値を与えられるものではないはずだ。そうすると機械にその「曖昧さ」を表現してもらう必要がある。

そこで新たな考え方として「背が高い　度合い」を導入することでこの曖昧さを機械に表現してもらうことになる。例えば上記の例題で言えば以下のような感じに。

- 165cm 以上の人は背が高い度合い 1
- 150cm 未満 人は背が高い度合い 0
- 150cm\~160cmの人は背が高い度合い 0.1~0.9

こうすることでたった0.1cmの違いによって背が高い/低いの分断が起こることを回避できるようになる。

## 仮説・モデル表現

我々の仮説は以下を満たすものとする。要はこの仮説が出力するのは確率であると言える。

$$0 \leq h_\theta(x) \leq 1$$

このとき「シグモイド関数」という新たな形式を取り入れると、仮説表現は以下のようになる。

- $ h_\theta(x) = g(\theta^Tx)$ : 任意の実数$\theta^Tx$の$g$という関数による写像が$h_\theta(x)$   
- $ z = \theta^Tx $ : 特徴量$\theta$とデータ$x$の積を$z$により抽象化  
- $ g(z) = \frac{1}{1 + e^{-z}} $ : $g$による$z$の写像がシグモイド関数またはロジスティック関数  

![https://en.wikipedia.org/wiki/Logistic_function](logistic-function.png)  

この$g(z)$はどんな実数も$\{0, 1\}$区間にマップするため任意の値を持つ関数を分類に適したより良い関数に変換することができる。1に近ければ1だし、0に近ければ0という容易な判断ができるようになった。

たとえば$g(z)$は結果が1と判断できるように確率$h_\theta(x) = 0.7 $を我々に与えてくれる。

結果が1もしくは0と判断できる確率は以下のように表現される。

$ h_\theta(x) = P(y = 1 | x; \theta) = 1 - P(y = 0 | x; \theta) $ ※変数$x$のパラメータが$\theta$である時出力$y$が1の確率を表す仮説  
$ P(y = 0 | x; \theta) + P(y = 1 | x; \theta) = 1 $ ※出力$y$が0もしくは1である確率の和は必ず1になる  

これはつまり予測結果が1の確率が70%ならば0である確率は30%であるということだ。

## 課題1. シグモイド関数を実装してください。
### ※YOUR CODE IS HERE

In [11]:
def sigmoid(X):
    """Sigmoid function
    
    Args:
        X(int/float or np.array): input data
    
    Returns:
        (int/float or np.array):
    """
    return 

### test cases

In [4]:
np.testing.assert_allclose(sigmoid(-5), 0.0066929, rtol=1e-05)
np.testing.assert_allclose(sigmoid(0), 0.50000, rtol=1e-05)
np.testing.assert_allclose(sigmoid(5), 0.99331, rtol=1e-05)

np.testing.assert_allclose(
    sigmoid(np.array([4, 5, 6])), np.array([0.98201, 0.99331, 0.99753]), rtol=1e-05)

np.testing.assert_allclose(
    sigmoid(np.array([[-1], [0], [1]])), np.array([[0.26894], [0.50000], [0.73106]]), rtol=1e-05)

## 決定境界


0か1かの分類結果を得るために我々は以下の仮説表現に従って得られた確率から0/1を判断することができる。

$$ h_\theta(x) \geq 0.5 \rightarrow y = 1 $$

$$ h_\theta(x) < 0.5 \rightarrow y = 0 $$

$ g(z) = \frac{1}{1 + e^{-z}} $はその入力が0以上であるときその出力が0.5以上となるなど、その振る舞いは以下のように定義される。

- $ z = 0, e^0 = 1 \Rightarrow g(z) = 1/2 $
- $ z \rightarrow \infty, e^{-\infty} \rightarrow 0 \Rightarrow g(z) = 1 $
- $ z \rightarrow -\infty, e^{\infty} \rightarrow \infty \Rightarrow g(z) = 0 $

つまりgに対する入力が$\theta^TX$ならば

$$h_\theta(x) = g(\theta^Tx) \geq 0.5 \ \ when \ \ \theta^Tx \geq 0 $$

よって以下が言える

$$\theta^Tx \geq 0 \Rightarrow y = 1 $$
$$\theta^Tx \leq 0 \Rightarrow y = 0 $$

そして決定境界は$y=0$と$y=1$となるエリアを分割する線のことであり、以下のような仮説関数(e.g. $z = \theta_0 + \theta_1x_1+\theta_2x_2$)によって定義される。

$$ \theta = \begin{bmatrix} 5 \\ -1 \\ 0 \end{bmatrix}$$
$$ y = 1 \ \ if \ \  5 + (-1)x_1 + 0x_2 \geq 0 $$
$$ 5 - x_1 \geq 0 $$
$$ x_1 \leq 5 $$

この場合、決定境界は垂直な直線で$x_1 = 5$で表現され、その左は全て$y = 1$右は$y=0$となる。

そしてシグモイド関数$g(z)$への入力は線形である必要がなく円形（e.g. $z = \theta_0 + \theta_1x_1^2+\theta_2x_2^2$）のような形でも構わない。

## コスト関数

ここでも線形回帰とは異なる関数を使う必要がある。何故ならばロジスティック関数の出力は波状であり多くの局所最適解が存在するため、非線形関数の二乗和は非凸（局所最適解がたくさんある）になる可能性があるからである。つまり線形回帰におけるコスト関数をロジスティック関数に適用すると凸関数であることが保証できないということだ。

代わりに以下のコスト関数を使う。

$$J(\theta) = \frac{1}{m} \sum_{i=1}^{m}Cost(h_\theta(x^{(i)}), y^{(i)})$$　　
$$ Cost(h_\theta(x), y) = -log(h_\theta(x)) \ \ if \ \ y=1 $$　　
![y=1](lr-cost-function-y1.png)  
$$ Cost(h_\theta(x), y) = -log((1 - h_\theta(x)) \ \ if \ \ y=0 $$　　
![y=0](lr-cost-function-y0.png)

このコスト関数の見方は仮説の予測結果がyから離れれば離れるほどコスト関数の出力は大きくなる。逆に予測結果がyに等しいとコスト関数の出力は0に近づく。

$$ Cost(h_\theta(x), y) = 0 \ \ if \ \ h_\theta(x) = y $$

また$y$が0の場合、仮説関数も0を出力すればコスト関数は0になります。仮説が1に近づけば、コスト関数は無限大に近づくことになります。

$$ Cost(h_\theta(x), y) \rightarrow \infty \ \ if \ \ y=0 \ and \ h_\theta(x) \rightarrow 1 $$

$y$ が 1の場合、仮説関数が1を出力すればコスト関数は 0 になります。我々の仮説が 0 に近づけば、コスト関数は無限大に近づきます。

$$ Cost(h_\theta(x), y) \rightarrow \infty \ \ if \ \ y=1 \ and \ h_\theta(x) \rightarrow 0 $$

## 単純化したコスト関数と勾配効果法

これまでコスト関数を二つに場合分けしていたが、以下のように一つにまとめることができる。

$$ Cost(h_\theta(x), y) = -ylog(h_\theta(x)) - (1 - y)log(1 - h_\theta(x)) $$

上式から$y$が1の時は第二項$(1 - y)log(1 - h_\theta(x))$は0となり、もし$y$が0の時は第一項$-ylog(h_\theta(x))$が0になることに注目されたい。

以上よりコスト関数全体を以下のように表現することができる。

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}log(h_\theta(x^{(i)})) + (1-y^{(i)})log(1-h_\theta(x^{(i)}))]$$

### なぜこのコスト関数なのか？

そもそもなぜこういったコスト関数に少し違う角度から紐解いていく。なぜ両辺の対数を取るのかは置いておいて、まず冒頭の振り返りから。

$$ h_\theta(x) = P(y = 1 | x; \theta) = 1 - P(y = 0 | x; \theta) $$  
$$ P(y = 0 | x; \theta) + P(y = 1 | x; \theta) = 1 $$  

これは以下のように書き換えることができる。

$P(Y = y | X = x)  = P(Y = 1 | X = x)^y P(Y = 0 | X = x)^{1-y} = h_{\theta}(x)^y (1 - h_{\theta}(x))^{1-y}$

次に特徴量行列$X$とラベルベクトルyが与えられた時には、その$X$から$y$が生じる確率を考える。行列$X$の$i$行目のサンプルがラベル$y_i$に分類される確率を全て掛け算したものとして考えることができ、次のように表せられる。

$P(y|X) = \prod_{i=1}^{m} \left [ h_{\theta}(x^{(i)})^{y^{(i)}} (1 - h_{\theta}(x^{(i)}))^{1-y^{(i)}} \right ]$

この確率を最大化することを考えるが、掛け算の形式だと考えづらい（らしい）ので対数をとってマイナスをつけたものをコスト関数として扱う。

$-logP(y|X) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}log(h_\theta(x^{(i)})) + (1-y^{(i)})log(1-h_\theta(x^{(i)}))]$

### 勾配降下法

勾配降下法の一般的な式は以下のようなものであったことを思い出してほしい。

$$ Repeat\ \  {\theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\theta)} $$

微分の部分を計算すると、次のようになる。

$$\begin{align*}
& Repeat \; \lbrace \newline
& \; \theta_j := \theta_j - \frac{\alpha}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \newline & \rbrace
\end{align*}$$

これは線形回帰で使用していたものと同じであり、$\theta$の全ての値を同時に更新する必要がある。

ベクトル化による実装は以下のようになる。

$$ \theta := \theta - \frac{\alpha}{m}X^T(g(X\theta) - \overset{\rightarrow}{y}) $$

まずシグモイド関数の微分を計算します（これは$J(\theta)$の偏微分を求めるときに役立ちます）。

$\begin{align*}\sigma(x)'&=\left(\frac{1}{1+e^{-x}}\right)'=\frac{-(1+e^{-x})'}{(1+e^{-x})^2}=\frac{-1'-(e^{-x})'}{(1+e^{-x})^2}=\frac{0-(-x)'(e^{-x})}{(1+e^{-x})^2}=\frac{-(-1)(e^{-x})}{(1+e^{-x})^2}=\frac{e^{-x}}{(1+e^{-x})^2} \newline &=\left(\frac{1}{1+e^{-x}}\right)\left(\frac{e^{-x}}{1+e^{-x}}\right)=\sigma(x)\left(\frac{+1-1 + e^{-x}}{1+e^{-x}}\right)=\sigma(x)\left(\frac{1 + e^{-x}}{1+e^{-x}} - \frac{1}{1+e^{-x}}\right)=\sigma(x)(1 - \sigma(x))\end{align*}$

これで、結果として得られる偏微分を求める準備ができました。

$\begin{align*}\frac{\partial}{\partial \theta_j} J(\theta) &= \frac{\partial}{\partial \theta_j} \frac{-1}{m}\sum_{i=1}^m \left [ y^{(i)} log (h_\theta(x^{(i)})) + (1-y^{(i)}) log (1 - h_\theta(x^{(i)})) \right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} \frac{\partial}{\partial \theta_j} log (h_\theta(x^{(i)}))   + (1-y^{(i)}) \frac{\partial}{\partial \theta_j} log (1 - h_\theta(x^{(i)}))\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \frac{\partial}{\partial \theta_j} h_\theta(x^{(i)})}{h_\theta(x^{(i)})}   + \frac{(1-y^{(i)})\frac{\partial}{\partial \theta_j} (1 - h_\theta(x^{(i)}))}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \frac{\partial}{\partial \theta_j} \sigma(\theta^T x^{(i)})}{h_\theta(x^{(i)})}   + \frac{(1-y^{(i)})\frac{\partial}{\partial \theta_j} (1 - \sigma(\theta^T x^{(i)}))}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \sigma(\theta^T x^{(i)}) (1 - \sigma(\theta^T x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{h_\theta(x^{(i)})}   + \frac{- (1-y^{(i)}) \sigma(\theta^T x^{(i)}) (1 - \sigma(\theta^T x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} h_\theta(x^{(i)}) (1 - h_\theta(x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{h_\theta(x^{(i)})}   - \frac{(1-y^{(i)}) h_\theta(x^{(i)}) (1 - h_\theta(x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} (1 - h_\theta(x^{(i)})) x^{(i)}_j - (1-y^{(i)}) h_\theta(x^{(i)}) x^{(i)}_j\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} (1 - h_\theta(x^{(i)})) - (1-y^{(i)}) h_\theta(x^{(i)}) \right ] x^{(i)}_j \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} - y^{(i)} h_\theta(x^{(i)}) - h_\theta(x^{(i)}) + y^{(i)} h_\theta(x^{(i)}) \right ] x^{(i)}_j \newline&= - \frac{1}{m}\sum_{i=1}^m \left [ y^{(i)} - h_\theta(x^{(i)}) \right ] x^{(i)}_j  \newline&= \frac{1}{m}\sum_{i=1}^m \left [ h_\theta(x^{(i)}) - y^{(i)} \right ] x^{(i)}_j\end{align*}$

このベクトル化実装は以下のようになる。

$$  \nabla J(\theta) = \frac{1}{m} \cdot X^T \cdot (g(X \cdot \theta) - \overset{\rightarrow}{y})  $$

## 課題3. コスト関数と勾配降下法を実装してください。
### ※YOUR CODE IS HERE

In [15]:
def lrCostFunction(X, y, theta):
    """Cosft function for logistic regression
    
    Args:
        - X(np.array): Input values
        - y(np.array): Actual output values
        - theta(np.array): parameters of hypothesis
    
    Returns:
        J(np.array): cost value
        grad(np.array): gradient value
    """
    return J, grad

def gradientDescent(X, y, theta, alpha, iterations=1000):
    """calculate gradient descent
    
    Args:
        - X(np.array): Input values
        - y(np.array): Actual output values
        - theta(np.array): parameters of hypothesis
        - alpha(float): learning rate
        - iterations(int): number of iterations
    
    Returns:
        - theta_min(np.array): parameters of hypothesis which make minimize cost function 
        - j_hist(list): history of cost "j"
    """
    return theta, j_history 

### test cases(only lrCostfunction)

In [16]:
theta = np.array([[-2],[-1], [1], [2]])
X = np.array([
    [0.1, 0.6, 1.1],
    [0.2, 0.7, 1.2],
    [0.3, 0.8, 1.3],
    [0.4, 0.9, 1.4],
    [0.5, 1.0, 1.5],
])
X = np.hstack((np.ones((5, 1)), X))
y = np.array([[1], [0], [1], [0], [1]])

J, grad = lrCostFunction(X, y, theta)

np.testing.assert_allclose(J, np.array([0.73482]), rtol=1e-05)
np.testing.assert_allclose(
    grad, np.array([[0.14656137],[0.05144159],[0.12472227],[0.19800296]]), rtol=1e-05)

## ここまででex2-wisconsin-diagnostic-breast-cancerの演習に一旦移動

## 更なる最適化、結局出来合いのソルバーを使おうという話

「共役勾配」、「BFGS」、「L-BFGS」は、勾配降下法の代わりに使用できる、より洗練された高速なθの最適化方法です。勾配効果法は更新レートやイテレーションといった調整項があり扱いづらいので、先ほどの演習で突如現れたニュートン法はそのような調整項がないことがわかりやすい利点だ。さらに目的関数の二回微分計算しながら最適解を探すため勾配降下法に比べてある条件下においては収束しやすい。しかしながら諸々の数学的な理由（二回微分が正定値性を満たさない場合）を考慮するとこれも不十分であり・・・。といった具合に数学シロウトの我々を深淵が待ち受けている。これ以上は学習コストとその見返りのバランスが悪いとの判断から問題に応じてscipyの最適化ソルバーを使っていく方針に切り替えた。（2021/10/13現在）

今回使用するのは[scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)、デフォルトではBFGSを使うことになるだろう。BFGSは[準ニュートン法](https://ja.wikipedia.org/wiki/%E6%BA%96%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%88%E3%83%B3%E6%B3%95)とも呼ばれているようで、ニュートン法で都度計算される[ヘッセ行列](https://ja.wikipedia.org/wiki/%E3%83%98%E3%83%83%E3%82%BB%E8%A1%8C%E5%88%97)（要は二階微分）の近似行列$B_k$を用いることでその正定値性（おそらく下に凸であること）を保証しながらパラメーターの更新をすることが可能だ。

## 正則化

オーバーフィッティングの問題

正則化は、オーバーフィッティングの問題に対処するために設計されている。

ハイバイアスやアンダーフィッティングとは、仮説関数hの形がデータの傾向にうまく対応していない場合をいう。これは通常、関数が単純すぎたり、使用する特徴が少なすぎたりすることが原因です。例えば、$h_\theta(x) = \theta_0 + \theta_1x_1 + \theta_2x_2$とすると 、線形モデルがトレーニングデータによく適合し、一般化できるという初期の仮定をしていますが、実際にはそうではないかもしれません。

一方、オーバーフィッティングや高分散は、仮説関数が利用可能なデータにフィットしても、新しいデータを予測するためにうまく一般化できないことが原因で起こります。通常は、データとは関係のない不必要な曲線や角度をたくさん作るような複雑な関数が原因となります。

この用語は、線形回帰とロジスティック回帰の両方に適用されます。オーバーフィッティングの問題に対処するには、主に2つの選択肢があります。

1) 特徴量の数を減らす。

       a) 残すべき特徴を手動で選択する。

       b) モデル選択アルゴリズムを使用する（今回は除外）。

2) 正則化

すべての特徴量を残すが，パラメータ $\theta_j$の重みを減らします。正則化は、わずかに有用な特徴がたくさんある場合に有効です。

## コスト関数
仮説関数のオーバーフィッティングが発生した場合、関数内のいくつかの項のコストを増やすことで、その項が持つ重みを減らすことができます。

例えば，次のような関数をより二次関数的にしたいとします．

$ \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3 + \theta_4 x^4 $

ここで$\theta_3x^3$と$\theta_4x^4$の影響を排除したいと思います。これらの特徴量を取り除いたり仮説の形を変えたりすることなく、代わりに以下のような形でコスト関数を修正することができます。

$ min_{\theta} \frac{1}{2m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})^2 + 1000 \cdot \theta_3^2 + 1000 \cdot \theta_4^2$

$\theta_3$と$\theta_4$のコストを膨らませるために余分に二つの項を追加しました。ここでコスト関数をゼロに近づけるためには$\theta_3$と$\theta_4$をゼロに近い値にする必要があります。これは$\theta_3x^3$と$\theta_4x^4$を大きく減らすことを意味します。

また、すべてのシータ・パラメータを単一の和で正則化することもできます。

$ min_{\theta} \frac{1}{2m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})^2 + \lambda \sum_{j=1}^{n} \theta_{j}^{2}$

λ(ラムダ)は正則化パラメータです。シータパラメータのコストをどれだけ膨らませるかを決定します。正則化の効果をこのインタラクティブなプロットで視覚化することができます。

https://www.desmos.com/calculator/1hexc8ntqp

上記のコスト関数に余分な和を加えることで、仮説関数の出力を平滑化し、オーバーフィッティングを減らすことができます。lambdaを大きくしすぎると，関数を平滑化しすぎてアンダーフィッティングを引き起こす可能性がある．

## 正則化ロジスティック回帰

ロジスティック回帰の正則化は、線形回帰の正則化と同様の方法で行うことができます。まずは、コスト関数から始めましょう。

コスト関数
ロジスティック回帰のコスト関数が次のようだったことを思い出してください。

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}log(h_\theta(x^{(i)})) + (1-y^{(i)})log(1-h_\theta(x^{(i)}))]$$

この等式を一つの項を加えて正則化することができる。

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}log(h_\theta(x^{(i)})) + (1-y^{(i)})log(1-h_\theta(x^{(i)}))]　+ \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2$$

上式の第二項$ \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2$はバイアス項である$\theta_0$を除外することを示している。例えば$\theta$ベクトルは0からnでインデックス化されている（$\theta_0$から$\theta_n$までn+1の値を持つ）、そしてその合計は1からnまでの計算をすることで$\theta_0$を明示的にスキップしている。

#### 勾配降下法

線形回帰と同様に$\theta_0$と残りのパラメータを**別々に**アップデートしよう。なぜなら$\theta_0$は先述の理由から正則化されないからだ。

Repeat {  
$\theta_0 := \theta_0 - \alpha \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_0^{(i)}$
$\theta_j := \theta_0 - \alpha \left[ \left( \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \right) + \frac{\lambda}{m} \theta_j \right] \ \ \ j \in \{1, 2, \dots n \}$  
}

これは、線形回帰で紹介した勾配降下関数と同じです。

## 課題5. 正則化コスト関数を実装してください。

In [17]:
def lrCostFunctionReg(X, y, theta, lam):
    """Cosft function for logistic regression
    
    Args:
        - X(np.array): Input values
        - y(np.array): Actual output values
        - theta(np.array): parameters of hypothesis
        - alpha(float): Regularization strength
    
    Returns:
        J(np.array): cost value
        grad(np.array): gradient value
    """
    return J, grad

theta = np.array([[-2], [-1], [1], [2]])
X = np.array([[1., 0.1, 0.6, 1.1], [1., 0.2, 0.7, 1.2], [1., 0.3, 0.8, 1.3], [1., 0.4, 0.9, 1.4], [1., 0.5, 1., 1.5]])
y = np.array([[1], [0], [1], [0], [1]])
alpha = 3

J, grad = lrCostFunctionReg(X, y, theta, alpha)

np.testing.assert_allclose(J, np.array([2.534819]), rtol=1e-06)
np.testing.assert_allclose(grad, np.array([[0.146561], [-0.54856], [0.72472], [1.39800]]), rtol=1e-05)

# 本講習から除外した項目
- 多クラス分類